In [ ]:
# Punto 1) Cargue la base de datos Machines.csv a un DataFrame

import pandas as pd

df = pd.read_csv("Machines.csv")  # si estás en Colab, sube el archivo y ajusta la ruta
display(df.head())
print("Shape:", df.shape)

In [ ]:
# Punto 2) Proporción de cada clase de Failure y balanceo

proporciones = df["Failure"].value_counts(normalize=True).sort_index()
conteos = df["Failure"].value_counts().sort_index()

display(pd.DataFrame({"conteo": conteos, "proporción": proporciones}))

p_min = proporciones.min()
if p_min < 0.4:
    print("Conclusión: el conjunto está DESBALANCEADO (al menos una clase < 40%).")
else:
    print("Conclusión: el conjunto está BALANCEADO (todas las clases ≥ 40%).")

print("Nota: aunque no sea extremo, 68/32 suele considerarse desbalance moderado.")

In [ ]:
# Punto 3) Defina X (predictoras) y y (objetivo)

X = df.drop(columns=["Failure"])
y = df["Failure"]

print("Predictoras (X):", list(X.columns))
print("Objetivo (y): Failure")

In [ ]:
# Punto 4) Train/Test split 80/20 con estratificación (random_state=42)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Shapes -> X_train:", X_train.shape, "X_test:", X_test.shape)
print("Proporciones en train:", y_train.value_counts(normalize=True).to_dict())
print("Proporciones en test :", y_test.value_counts(normalize=True).to_dict())

In [ ]:
# Punto 5) Estandarice variables predictoras

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)  # se ajusta SOLO con train
X_test_sc  = scaler.transform(X_test)

print("Media (train, aprox 0):", X_train_sc.mean(axis=0).round(3))
print("Std  (train, aprox 1):", X_train_sc.std(axis=0).round(3))

In [ ]:
# Punto 6) Genere modelos: SVM y Árbol de Decisión

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

svm_model = SVC(kernel="rbf", C=1.0, gamma="scale")  # baseline razonable
tree_model = DecisionTreeClassifier(random_state=42)  # baseline

svm_model.fit(X_train_sc, y_train)
tree_model.fit(X_train_sc, y_train)

print("Modelos entrenados: SVM y Decision Tree")

In [ ]:
# Punto 7) Métricas + matriz de confusión para cada modelo

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score
)
import numpy as np

def evaluar_modelo(nombre, model, Xte, yte, usa_decision_function=False):
    y_pred = model.predict(Xte)
    cm = confusion_matrix(yte, y_pred)  # orden: [[TN, FP],[FN, TP]] si clases 0/1
    out = {
        "modelo": nombre,
        "accuracy": accuracy_score(yte, y_pred),
        "precision": precision_score(yte, y_pred, zero_division=0),
        "recall": recall_score(yte, y_pred),
        "f1": f1_score(yte, y_pred),
        "TN": cm[0,0], "FP": cm[0,1], "FN": cm[1,0], "TP": cm[1,1],
    }
    # ROC AUC (opcional pero útil)
    if usa_decision_function:
        scores = model.decision_function(Xte)
        out["roc_auc"] = roc_auc_score(yte, scores)
    else:
        proba = model.predict_proba(Xte)[:,1] if hasattr(model, "predict_proba") else None
        out["roc_auc"] = roc_auc_score(yte, proba) if proba is not None else np.nan

    print(f"\n=== {nombre} ===")
    print("Matriz de confusión [ [TN FP], [FN TP] ]:\n", cm)
    print("\nReporte de clasificación:\n", classification_report(yte, y_pred, digits=4))
    return out

resultados = []
resultados.append(evaluar_modelo("SVM (RBF)", svm_model, X_test_sc, y_test, usa_decision_function=True))
resultados.append(evaluar_modelo("Decision Tree", tree_model, X_test_sc, y_test, usa_decision_function=False))

res_df = pd.DataFrame(resultados).set_index("modelo")
display(res_df[["accuracy","precision","recall","f1","roc_auc","TN","FP","FN","TP"]].round(4))

In [ ]:
# Punto 8) ¿Qué modelo es mejor y por qué? (basado en métricas)

# Regla práctica: en mantenimiento suele pesar fuerte Recall/F1 del 'Failure=1'.
# Aquí hacemos una comparación simple por F1 y ROC-AUC.
mejor_por_f1 = res_df["f1"].idxmax()
mejor_por_auc = res_df["roc_auc"].idxmax()

print("Mejor por F1 :", mejor_por_f1, "-> F1 =", float(res_df.loc[mejor_por_f1, "f1"]))
print("Mejor por AUC:", mejor_por_auc, "-> AUC =", float(res_df.loc[mejor_por_auc, "roc_auc"]))

if mejor_por_f1 == mejor_por_auc:
    print(f"Conclusión: {mejor_por_f1} es el mejor modelo global (mejor F1 y mejor AUC).")
else:
    print("Conclusión: depende de la métrica priorizada. Si priorizas F1 el mejor es:", mejor_por_f1,
          "y si priorizas AUC el mejor es:", mejor_por_auc)

# Comentario típico: SVM suele generalizar mejor que un árbol sin poda/tuning en datos continuos estandarizados.

In [ ]:
# Punto 9) Mantenimiento: ¿Qué métrica es más importante?

# Suposición estándar: 'Failure=1' significa falla (evento raro y costoso).
# En mantenimiento predictivo, perder una falla (FN) suele ser más caro que una alarma extra (FP).
# Por eso, la métrica clave suele ser RECALL (sensibilidad) de la clase Failure=1.

for modelo in res_df.index:
    recall = float(res_df.loc[modelo, "recall"])
    fn = int(res_df.loc[modelo, "FN"])
    print(f"{modelo}: Recall(Failure=1) = {recall:.4f} | FN = {fn}")

print("\nRespuesta: la métrica más importante suele ser el RECALL de Failure=1 (minimiza falsos negativos).")

In [ ]:
# Punto 10) Mantenimiento: ¿qué error es más grave, FP o FN?

# Tomamos el mejor modelo por F1 para ilustrar.
best = mejor_por_f1
fp = int(res_df.loc[best, "FP"])
fn = int(res_df.loc[best, "FN"])

print("Modelo ilustrativo:", best)
print("FP (falsos positivos):", fp, "-> alarma de falla cuando no falla")
print("FN (falsos negativos):", fn, "-> NO detectas una falla real")

print("""\nRespuesta: en mantenimiento suele ser más grave el FALSO NEGATIVO (FN).
Justificación: predices 'no falla' pero sí falla -> paros no planificados, daño a equipos, riesgos de seguridad y costos altos.
Un FP normalmente solo implica una inspección/mantenimiento extra (costo menor y controlable).""")